<a href="https://www.kaggle.com/code/prashantsparhad/day-29-machine-learning-without-pipelines?scriptVersionId=101234211" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/day20-1/train.csv


In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

In [3]:
df=pd.read_csv("/kaggle/input/day20-1/train.csv")

In [4]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df.drop(columns=["PassengerId","Name","Ticket","Cabin"],inplace=True)

In [6]:
df.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [7]:
X_train,X_test,y_train,y_test=train_test_split(df.iloc[:,1:],df.iloc[:,0:1],
                                              test_size=0.2,
                                              random_state=42)

In [8]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [9]:
y_train.head()

,Survived
331,0
733,0
382,0
704,0
813,0


In [10]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [11]:
#Applying SimpleImputer

si_age=SimpleImputer()
si_embarked=SimpleImputer(strategy="most_frequent")

X_train_age=si_age.fit_transform(X_train[["Age"]])
X_train_embarked=si_embarked.fit_transform(X_train[["Embarked"]])

X_test_age=si_age.fit_transform(X_test[["Age"]])
X_test_embarked=si_embarked.fit_transform(X_test[["Embarked"]])

In [12]:
#one hot encoding for sex and Embarked column

ohe_sex=OneHotEncoder(sparse=False,handle_unknown="ignore")
ohe_embarked=OneHotEncoder(sparse=False,handle_unknown="ignore")

X_train_sex=ohe_sex.fit_transform(X_train[["Sex"]])
X_train_embarked=ohe_embarked.fit_transform(X_train_embarked)

X_test_sex=ohe_sex.fit_transform(X_test[["Sex"]])
X_test_embarked=ohe_embarked.fit_transform(X_test_embarked)

In [13]:
X_train_embarked

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [14]:
X_train_rem=X_train.drop(columns=["Sex","Age","Embarked"])

In [15]:
X_test_rem=X_test.drop(columns=["Sex","Age","Embarked"])

In [16]:
X_train_transformed=np.concatenate((X_train_rem,X_train_age,X_train_sex,X_train_embarked),axis=1)
X_test_transformed=np.concatenate((X_test_rem,X_test_age,X_test_sex,X_test_embarked),axis=1)

In [17]:
X_test_rem.shape,X_test_age.shape,X_test_sex.shape,X_test_embarked.shape

((179, 4), (179, 1), (179, 2), (179, 3))

In [18]:
X_train_transformed.shape,X_test_transformed.shape

((712, 10), (179, 10))

In [19]:
clf=DecisionTreeClassifier()
clf.fit(X_train_transformed,y_train)

DecisionTreeClassifier()

In [20]:
y_pred=clf.predict(X_test_transformed)
y_pred

array([0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 1])

In [21]:
from sklearn.metrics import accuracy_score

In [22]:
accuracy_score(y_pred,y_test)

0.7430167597765364

In [23]:
import pickle

In [24]:
pickle.dump(ohe_sex,open("ohe_sex.pkl","wb"))
pickle.dump(ohe_embarked,open("ohe_embarked.pkl","wb"))
pickle.dump(clf,open("clf.pkl","wb"))

In [25]:
import pickle
import numpy as np

In [26]:
ohe_sex=pickle.load(open("ohe_sex.pkl","rb"))
ohe_embarked=pickle.load(open("ohe_embarked.pkl","rb"))
clf=pickle.load(open("clf.pkl","rb"))

In [27]:
test_input=np.array([2,"male",31.0,0,0,10.5,"S"],dtype=object).reshape(1,7)

In [28]:
test_input

array([[2, 'male', 31.0, 0, 0, 10.5, 'S']], dtype=object)

In [29]:
test_input_sex=ohe_sex.transform(test_input[:,1].reshape(1,1))

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  "X does not have valid feature names, but"


In [30]:
test_input_sex

array([[0., 1.]])

In [31]:
test_input_embarked=ohe_embarked.transform(test_input[:,-1].reshape(1,1))

In [32]:
test_input_embarked

array([[0., 0., 1.]])

In [33]:
test_input_age = test_input[:,2].reshape(1,1)

In [34]:
test_input_age

array([[31.0]], dtype=object)

In [35]:
test_input_transformed = np.concatenate((test_input[:,[0,3,4,5]],test_input_age,test_input_sex,test_input_embarked),axis=1)

In [36]:

test_input_transformed.shape

(1, 10)

In [37]:
clf.predict(test_input_transformed)

array([1])